### Service Disruption Project

**This project was assigned to us, not to find the prediction, but to practice on cleaning and prepping Data.** 

**The data provided to us was for a telecom project where they was a prediction of which remote service areas would need repairs. Some of the service areas were in very remote areas and if one would stop working that would mean many of their customers would be without service tarnishing their reputation and losing customer. The goal was to fix service areas that were predicted to fail before they would fail in order prevent disruption.**

**I handeled this project slightly different than the rest of my classmates. I started out by reading in each cvs file and striping the the letters from the value. This resulted in just leaving a number behind. Note* just because I removed the letters does not automatically turn the column from a string to an int.* I left the columns at strings because I turn them into dummies later on. *If you choose to turn the column into an **int** you must use astype(int) after stripping it.**

In [1]:
import pandas as pd
import numpy as np

In [2]:
event = pd.read_csv('event_type.csv')
event['event_type'] = event['event_type'].str.strip("event_type")
event.reset_index()
event.head()

,id,event_type
0,6597,11
1,8011,15
2,2597,15
3,5022,15
4,5022,11


In [3]:
log_feature = pd.read_csv('log_feature.csv')
log_feature['log_feature'] = log_feature['log_feature'].str.strip("feature")
log_feature.head()

,id,log_feature,volume
0,6597,68,6
1,8011,68,7
2,2597,68,1
3,5022,172,2
4,5022,56,1


In [4]:
resource_type = pd.read_csv('resource_type.csv')
resource_type['resource_type'] = resource_type['resource_type'].str.strip('resource_type ')
resource_type.head()

,id,resource_type
0,6597,8
1,8011,8
2,2597,8
3,5022,8
4,6852,8


In [5]:
severity_type = pd.read_csv('severity_type.csv')
severity_type['severity_type']=severity_type['severity_type'].str.strip('severity_type ')
severity_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18552 entries, 0 to 18551
Data columns (total 2 columns):
id               18552 non-null int64
severity_type    18552 non-null object
dtypes: int64(1), object(1)
memory usage: 290.0+ KB


In [6]:
train = pd.read_csv('train.csv')
train['location'] = train['location'].str.strip('location ')
train.head()

,id,location,fault_severity
0,14121,118,1
1,9320,91,0
2,14394,152,1
3,8218,931,1
4,14804,120,0


**Once I cleaned up the data. I merged the columns together. As you can tell I did not merge the column "fault_severity". I did this because "fault_severity" will be the target variable. All other columns were merged together and assigned to the variable df. "fault_severity" was assigned to variable severity.**

In [7]:
data = event.merge(log_feature)
data = data.merge(resource_type)
data = data.merge(severity_type)
data = data.merge(train[['id','location']])
severity = train[['fault_severity']]
data.head()

,id,event_type,log_feature,volume,resource_type,severity_type,location
0,8011,15,68,7,8,2,1
1,2588,15,82,9,8,1,1
2,2588,15,201,5,8,1,1
3,2588,15,80,15,8,1,1
4,2588,15,203,5,8,1,1


In [8]:
severity.tail()

,fault_severity
7376,0
7377,0
7378,2
7379,0
7380,0


**Now that both data frames have been put togther; I create dummies out of the df dataframe. I then group it by "id".** 

In [9]:
df = pd.get_dummies(data)
df = df.groupby('id').sum()


In [10]:
df.head()

,volume,event_type_ 1,event_type_ 10,event_type_ 11,event_type_ 12,event_type_ 13,event_type_ 14,event_type_ 15,event_type_ 18,event_type_ 19,...,location_987,location_989,location_99,location_990,location_991,location_994,location_995,location_996,location_998,location_999
id,,,,,,,,,,,,,,,,,,,,,
1,20,0.0,0.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**To setup the data to use in algrothims I assign df to x and severity to y.** 

In [11]:
import statsmodels.api as sm
x = df
y = severity
x.shape, y.shape

C:\Users\YannellyS\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


((7381, 1325), (7381, 1))

**For the purpose of fitting and predicting I used Train Test Split.**

In [12]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.5, random_state=10)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

C:\Users\YannellyS\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


((3690, 1325), (3691, 1325), (3690, 1), (3691, 1))

In [13]:
from sklearn.metrics import accuracy_score

**SGDClassifed in linear_model gave me the best accuracy score of all the ones I tried.**

**If you are having trouble trying to figure out which algorithm to use, you can start by using TPOT "https://rhiever.github.io/tpot/installing/".**

In [14]:
from sklearn import linear_model
linear = linear_model.SGDClassifier()
linear = linear.fit(x_train, y_train)
linear_predict = linear.predict(x_test)
accuracy_score(y_test, linear_predict)

C:\Users\YannellyS\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6515849363316174

**As you can see below, Gradient Boosting Classifier did not yield a better accuracy score than SGDClassifier.**

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier()
GBC = GBC.fit(x_train,y_train)
GBC_predict = GBC.predict(x_test)
accuracy_score(y_test, GBC_predict)

C:\Users\YannellyS\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.63722568409645086

**First, Apply your prediction to the data set.**

**Second, saved the results in variable *disruption*.**

**Third, double check you shapes match up.**

In [16]:
y_predict= linear.predict(x)

In [17]:
disruption = data[['id']]
disruption = disruption.drop_duplicates()
disruption.shape, y_predict.shape

((7381, 1), (7381,))

**I then added a column *Disruption* to disruption converting it into a dataframe. In the column *Disruption* will be the predictions. **

In [18]:
disruption["Disruption"] = y_predict


In [19]:
disruption.head()


,id,Disruption
0,8011,0
1,2588,0
9,4848,0
19,6914,0
21,5337,0


**Saved disruptions into a new csv. I removed the index in this case.**

In [20]:
disruption.to_csv('disruptionfinaloutput.csv', index = False)